In [1]:
!pip install pytube torch torchvision torchaudio openai-whisper transformers


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\tqdm.exe' -> 'C:\\Python312\\Scripts\\tqdm.exe.deleteme'




     ---------------------------------------- 0.0/798.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/798.6 kB ? eta -:--:--
      --------------------------------------- 10.2/798.6 kB ? eta -:--:--
     ---- ---------------------------------- 92.2/798.6 kB 1.3 MB/s eta 0:00:01
     ----------------------- -------------- 491.5/798.6 kB 3.9 MB/s eta 0:00:01
     -------------------------------------  788.5/798.6 kB 5.0 MB/s eta 0:00:01
     -------------------------------------- 798.6/798.6 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ------------------

In [2]:
import os
from pytube import YouTube
import whisper

# Download YouTube Video and Extract Audio
def download_youtube_audio(url, output_path='audio.mp4'):
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    audio_file = video.download(filename=output_path)
    return audio_file

# Convert Speech to Text using Whisper
def speech_to_text(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result['text']

# Example usage
if __name__ == "__main__":
    youtube_url = 'https://youtu.be/X2huNCHDwMQ?si=XoHoOQWb7trbI09g'
    audio_path = download_youtube_audio(youtube_url)
    transcript = speech_to_text(audio_path)
    print("Transcript:", transcript)


ModuleNotFoundError: No module named 'pytube'

In [ ]:
!pip install torch

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, EncoderDecoderModel, BertTokenizer

def summarize_text(text, model_name="facebook/bart-large-cnn"):
    if model_name.startswith("facebook/bart"):
        tokenizer = BartTokenizer.from_pretrained(model_name)
        model = BartForConditionalGeneration.from_pretrained(model_name)
        inputs = tokenizer(text, max_length=1024, return_tensors='pt', truncation=True)
        summary_ids = model.generate(
            inputs['input_ids'],
            max_length=150,
            min_length=80,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    elif model_name.startswith("bert"):
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")
        inputs = tokenizer(text, max_length=512, return_tensors='pt', truncation=True)
        summary_ids = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=150,
            min_length=80,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    else:
        raise ValueError("Unsupported model_name. Choose 'facebook/bart-' or 'bert-'")

    return summary

if __name__ == "__main__":

    summary_bart = summarize_text(transcript, model_name="facebook/bart-large-cnn")
    print("Summary using BART:")
    print(summary_bart)

    summary_bertsum = summarize_text(transcript, model_name="bert-base-uncased")
    print("\nSummary using BERTSum:")
    print(summary_bertsum)


Summary using BART:
If you have zero views on your channel, your first video is not going to give you. Period. So stop sitting there and thinking for months and months on end and just get to work and start uploading. There's literally no such thing as a perfect video. Don't really worry about what your niche is like people get hung up on this like I don't know what to make videos about.

Summary using BERTSum:
this applies to people that not upload videos, but have dreams of being a youtuber. here's some amazing youtube advice from 10 very successful youtubers. you'll get into number one if you have zero views on your 101st video. you can't expect a perfect video, but you're not going to write a new one. there's literally no such thing as perfect video.
